In [ ]:
!py -3 -m pip install --upgrade Jinja2==3.0.3

In [ ]:
from datetime import datetime, timedelta
import os
from jinjasql import JinjaSql

In [ ]:
def read_contents_file(file_path, encoding_scheme='cp1252'):
    inputfile = open(file_path, 'r', encoding=encoding_scheme)
    return inputfile.read()

In [ ]:
#Write file to local directory
def write_file_local(path,file_data):
    
    with open(path, 'w') as file:
        file_string = file_data
        file.write(file_string)

In [ ]:
#path of file with table name and database name in csv file
ddlgen_template_path = "SQL_Templates"
lob = "PI"
lob_abbr = "PI"
project_abbr = "OPS"

In [ ]:
#get the DDLs for the tables from teradata 
def gen_bq_views_ddl():
    dt1 = datetime.now()

    try:
        lob_upper = lob.strip().upper()
        lob_upper_fmt = "'{}'".format(lob_upper)
        lob_abbr_lower = lob_abbr.strip().lower()
        project_abbr_upper = project_abbr.strip().upper()
        project_abbr_fmt = "'{}'".format(project_abbr_upper)
        starting_curly_braces = "{{"
        ending_curly_braces = "}}"
        monitoring_views = "{{ params.param_monitoring_views_dataset_name }}"
        authorized_views = "{{ params.param_auth_base_views_dataset_name }}"

        output_path_folder = "LOB\\"

        if not os.path.exists(f"{output_path_folder}{lob_upper}"):
            os.makedirs(f"{output_path_folder}{lob_upper}")

        for filename in os.listdir(ddlgen_template_path):
            file_relative_path = os.path.join(ddlgen_template_path, filename)
            DDL_GEN_TEMPLATE = read_contents_file(file_relative_path)
            match filename:
                case 'audit_control_combined.sql':
                    query = DDL_GEN_TEMPLATE.format(starting_curly_braces, lob_abbr_lower, ending_curly_braces,
                            lob_upper_fmt, starting_curly_braces, lob_abbr_lower, ending_curly_braces)
                case 'bq_query_cost.sql':
                    query = DDL_GEN_TEMPLATE.format(monitoring_views, project_abbr_fmt, authorized_views,
                            project_abbr_fmt, authorized_views)
                case 'bq_query_history.sql':
                    query = DDL_GEN_TEMPLATE.format(monitoring_views, project_abbr_fmt, authorized_views,
                            project_abbr_fmt, authorized_views)
                case 'composer_dag_status.sql':
                    query = DDL_GEN_TEMPLATE.format(monitoring_views, monitoring_views)
                case 'composer_task_status.sql':
                    query = DDL_GEN_TEMPLATE.format(monitoring_views, project_abbr_fmt, authorized_views)
            
            table_output_path = f"{output_path_folder}{lob_upper}\{filename}"
            write_file_local(table_output_path, query)

    except Exception as e1:
        print(e1)
        print("LOB : {}".format(lob))
        print("File Name : {}".format(filename))
        pass
    
    dt2 = datetime.now()
    print(dt2-dt1)

In [ ]:
print("Begin of Processing")

gen_bq_views_ddl()

print("End of Processing")